# Machine Learning with TensorFlow and BigQuery
In this notebook, we will use TensorFlow Python API to analyze data in BigQuery.

Presented at OC Advanced Analytics Meetup on June 20, 2017

Import all libraries

In [47]:
import google.datalab.ml as ml
import datalab.bigquery as bq
import tensorflow as tf

import tensorflow.contrib.learn as tfflow
from tensorflow.contrib import learn
from sklearn import metrics

import pandas as pd
import numpy as np
import shutil

Run query directly in BigQuery

In [48]:
%%sql
SELECT customerID, gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines,
InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV,
StreamingMovies, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges, Churn 
from [t3-dataintellence:public_misc_datasets.telco_customer_churn] limit 7000

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
9732-OUYRN,Female,0,True,False,49,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.7,False
0661-KQHNK,Female,0,True,True,6,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.5,False
4709-LKHYG,Female,0,True,True,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
9824-QCJPK,Male,0,True,False,36,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4716-MRVEN,Female,0,False,False,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.3,False
8084-OIVBS,Female,0,False,False,11,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,211.95,False
4323-ELYYB,Male,0,True,True,13,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,218.55,False
0967-BMLBD,Female,0,True,True,11,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,True,Mailed check,20.0,196.35,False
1269-FOYWN,Male,0,True,True,44,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Bank transfer (automatic),20.0,860.85,False
3556-BVQGL,Female,0,True,False,20,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,True,Mailed check,20.0,416.45,False


Query telco_customer_churn table from BigQuery and put the result in a dataframe

In [49]:
query = """
SELECT customerID, gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines,
InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV,
StreamingMovies, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, Cast(TotalCharges as float) as TotalCharges, Churn 
from [t3-dataintellence:public_misc_datasets.telco_customer_churn] limit 7000"""
df = bq.Query(query).to_dataframe()

Fill the empty fields with number 0. And then review the dataframe.

In [50]:
df = df.fillna(0)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9732-OUYRN,Female,0,True,False,49,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.70,False
1,0661-KQHNK,Female,0,True,True,6,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.50,False
2,4709-LKHYG,Female,0,True,True,29,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
3,9824-QCJPK,Male,0,True,False,36,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4,4716-MRVEN,Female,0,False,False,29,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.30,False


Convert all boolean column to numeric column, 1 for True and 0 for False

In [51]:
df['Partner'] = df['Partner'] .apply(lambda x: 1 if x else 0)
df['Dependents'] = df['Dependents'] .apply(lambda x: 1 if x else 0)
df['PhoneService'] = df['PhoneService'] .apply(lambda x: 1 if x else 0)
df['PaperlessBilling'] = df['PaperlessBilling'] .apply(lambda x: 1 if x else 0)
df['Churn'] = df['Churn'] .apply(lambda x: 1 if x else 0)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9732-OUYRN,Female,0,1,0,49,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,0,Credit card (automatic),19.0,918.70,0
1,0661-KQHNK,Female,0,1,1,6,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,0,Credit card (automatic),19.0,105.50,0
2,4709-LKHYG,Female,0,1,1,29,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,0,Electronic check,20.0,540.05,0
3,9824-QCJPK,Male,0,1,0,36,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,0,Mailed check,20.0,666.75,0
4,4716-MRVEN,Female,0,0,0,29,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,0,Mailed check,20.0,599.30,0


A column is called categorical if its value can only be one of the categories in a finite set. 

A column is called continuous if its value can be any numerical value in a continuous range. 

In [52]:
CONTINUOUS_COLUMNS = ["SeniorCitizen", "Partner", "Dependents", "tenure", "PhoneService", "PaperlessBilling", "MonthlyCharges", "TotalCharges"]
CATEGORICAL_COLUMNS = ["gender", "MultipleLines", "InternetService", "OnlineSecurity", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaymentMethod"]
LABEL_COLUMN = "Churn"

Create sparse columns for all categorical columns

Change this

| ID    | Gender |
|-------|--------|
| user1 | Male   |
| user2 | Female |
| user3 | Male   |

into this

| ID    | Gender_Male | Gender_Female |
|-------|-------------|---------------|
| user1 | 1           | 0             |
| user2 | 0           | 1             |
| user3 | 1           | 0             |


In [53]:
## Encode categorical features
dummies = pd.get_dummies(df[CATEGORICAL_COLUMNS])
CATEGORICAL_DUMMY_COLUMNS = list(dummies.columns.values)

## stack the original df with encoded categorical features 
df = pd.concat([df,dummies],axis=1)

Show all the created Sparse Columns

In [54]:
dummies.head()

,gender_Female,gender_Male,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
1,1,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
2,1,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,1,0
3,0,1,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,1
4,1,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,1


Show dataframe column types again

In [55]:
df.dtypes

customerID                                  object
gender                                      object
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                               object
InternetService                             object
OnlineSecurity                              object
OnlineBackup                                object
DeviceProtection                            object
TechSupport                                 object
StreamingTV                                 object
StreamingMovies                             object
Contract                                    object
PaperlessBilling                             int64
PaymentMethod                               object
MonthlyCharges                             float64
TotalCharges                   

Split the dataframe into three dataframes for training, and validation, testing

In [56]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

Usually we need to do some extra work to further prepare the dataset
1. correct data 
2. exclude outliers 
3. remove highly-correlated columns 
4. remove low variant columns 
5. standardize numeric columns

If we are doing deep learning, we also 
need to do
* batch normalization

Create feature columns and infer the real value features

In [57]:
FEATURE_COLUMNS = CATEGORICAL_DUMMY_COLUMNS+CONTINUOUS_COLUMNS
features = learn.infer_real_valued_columns_from_input(train[FEATURE_COLUMNS])

Try it with a linear classification model

In [58]:
#lC = tfflow.LinearClassifier(feature_columns=features)
#lC.fit(train[FEATURE_COLUMNS], train.Churn)

Try it with a deep nearul network classification model

* Train a DNN (Deep Neural Network) Classifier with
* hidden units[10, 20 ,10]

In [59]:
classifier = tfflow.DNNClassifier(hidden_units=[10, 20, 10],
                                  n_classes=2,
                                  feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f33669e7f50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Model Training steps(epoches) = 100 which can be tuned by early stopping

In [60]:
classifier.fit(train[FEATURE_COLUMNS], train.Churn, steps=100)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x7f33669e7e10>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=38, default_value=None, dtype=tf.float64, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x7f337e2851b8>, 'input_layer_min_slice_size': None})

A customer churn files are usually biased. For imbalanced data, we need to do
* ROC AUC

Validate model with AUC

In [61]:
val_score = metrics.roc_auc_score(validate.Churn, 
                              np.array([ pred_proba for pred_proba in classifier.predict_proba(validate[FEATURE_COLUMNS])])[:,1])
print("AUC: %f" % val_score)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
AUC: 0.761721


Model Prediction

In [62]:
test_preds = np.array([ pred_proba for pred_proba in classifier.predict_proba(test[FEATURE_COLUMNS])])[:,1]

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


In [63]:
pred_df = pd.DataFrame(test.Churn)
pred_df["PredictedChurn"] = test_preds
pred_df
#.to_csv("test_pred.csv", index=False)

,Churn,PredictedChurn
4850,0,0.427504
676,0,0.256444
5717,1,0.528952
4371,1,0.419019
4278,0,0.390704
4181,1,0.357943
6760,1,0.547727
595,0,0.220584
3136,0,0.177938
5473,1,0.523461
